In [117]:
%%time
#!/usr/bin/env python
"""Provides option table for NSE scrips.
"""
import requests
import lxml.html as LH
import pandas as pd
from bs4 import BeautifulSoup
import json
import datetime
import numpy as np

from IPython.display import display

from math import sqrt, exp, log, erf

pd.options.display.max_columns = None   # display all columns in jupyter

#### Declarations

num_trading_days_in_year = 252


#### URLs used
interest_url = "https://countryeconomy.com/bonds/india"
expiry_url = "https://www.nseindia.com/live_market/dynaContent/live_watch/fomwatchsymbol.jsp?key=NIFTY&Fut_Opt=Futures"
symbols_url = "https://www.5paisa.com/5pit/spma.asp"
dividend_url = "https://finance.google.com/finance?q=NSE:"

# NSE options-related URLs
nse_url_base = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
option_chain_url = nse_url_base + "option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol="
band_url = nse_url_base + "get_quote/GetQuote.jsp?symbol="
volatility_url = nse_url_base + "get_quote/GetQuoteFO.jsp?instrument=FUTSTK&underlying="

# Capture failed states
failed = pd.DataFrame([], columns = ['Function', 'Symbol', 'Underlying', 'Expiry', 'Error'])   # Catch the symbols with errors

def get_interest(url):
    """Returns interest rate as a float"""

    int_html = requests.get(url).content
    intrate = float(LH.fromstring(int_html).find_class('numero')[0].text)/100
    
    if intrate == 0.0:      # No interest rate!
        raise ValueError('Interest Rate cannot be 0 !!')   # Raise an exception
            
    return intrate

def get_symbols(url):
    """Returns equity scrips as a Series"""
    
    paisa = pd.read_html(url, header=0) [1]   # The second HTML table
    
    # Remove VIX and NIFTY 
    symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol

    # Replace & by %26 for NSE
    symbol.replace('&', '%26')

    # Sort the symbols
    symbol = symbol.sort_values(axis=0).reset_index(drop=True)
    
    if symbol.empty:
        raise ValueError('Symbols dataframe from paisa is empty!!')   # Raise an exception
    
    return symbol

def get_json(url, symbol):
    """Returns the NSE json dictionary"""
    
    json_url = url + symbol
    json_html = requests.get(json_url).text
    json_soup = BeautifulSoup(json_html, 'html.parser')
    data = json_soup.find(id='responseDiv').text.strip()
    json_dict = json.loads(data)['data'][0]
    
    return json_dict
    
    
def get_bands(symbol):
    """Returns bands and margin for the symbol as a dictionary"""

    try:
        d_band = get_json(band_url, symbol)
    except Exception as e:
        band = {'Symbol': symbol, 'low52' : np.nan, 'high52' : np.nan, 'cm_adj_low_dt': np.nan, 
                'cm_adj_high_dt': np.nan, 'pricebandlower':np.nan, 
                'pricebandupper': np.nan, 'applicableMargin': np.nan}
        return band
        
    # Extract relevant data from band dictionary
    band = {k: d_band[k] for k in ('low52', 'high52', 
                               'cm_adj_low_dt', 'cm_adj_high_dt', 
                               'pricebandlower', 'pricebandupper', 
                               'applicableMargin')}

#     band['Symbol'] = symbol
    
    return band

def get_daily_volatility(symbol):
    """Returns daily volatility as a dictionary"""
    
    vol_json = get_json(volatility_url, symbol)
    
    try:
        volatility = {k: vol_json[k] for k in ['dailyVolatility']}
    except Exception as e:
        volatility = {'dailyVolatility': np.nan}
    try:
        lot = {k: vol_json[k] for k in ['marketLot']}
    except Exception as e:
        lot = {'marketLot': np.nan}
        
    volatility.update(lot)
    
    return volatility

def get_dividend(symbol):
    """Returns the dividend as a float"""

    div_url = dividend_url + symbol
  
    page = requests.get(div_url)
        
    root = LH.fromstring(page.content)

    try:
        dividend = float(root.findall('.//table')[2].text_content().strip().split("\n")[2].split('/')[0])/100
    except Exception as e:
        dividend = {'dividend': np.nan}
        return dividend

    dividend = {'dividend': dividend}
    
    return dividend

def get_expiry_dates(url):
    """Returns expiry dates and DTE as a dataframe"""
    
    exp_html = requests.get(url).content
    fno_table = pd.read_html(exp_html, match='Expiry Date', header=0)[0]
    fno_expiry_series = pd.Series(fno_table['Expiry Date'])
    fno_expiry = pd.to_datetime(fno_expiry_series, format = "%d%b%Y")
    fno_expiry_upper = fno_expiry.dt.strftime("%d%b%Y").str.upper()   # converts to uppercase
    
    if fno_expiry.empty:
        raise ValueError('Expiry Dates are empty!!')   # Raise an exception
 
    dte = fno_expiry - datetime.datetime.now()
    dte = dte.rename("DTE")
    
    expiry = pd.concat([fno_expiry_upper, dte.dt.days], axis=1)
    
    return expiry

def get_option_chain(symbol, expiry, dte):
    """Returns the option chain as a dataframe"""
    
    u = option_chain_url + symbol + '&date=' + expiry
    
    chainhtml = requests.get(u).content
    chain = pd.read_html(chainhtml)[1][:-1]  # read the first table and drop the total
    chain.columns=chain.columns.droplevel(0) # drop the first row of the header
    chain = chain.drop('Chart', 1)           # drop the charts
    
    # Get the underlying stock price
    underlyingtbl = pd.read_html(chainhtml, match='Underlying Stock:')[0][1]
    underlying = underlyingtbl.iloc[0]
    stockprice = float(underlying.split(' ')[3])
    
    # Prepare the return
    chain["Symbol"] = symbol
    chain["Expiry"] = expiry
    chain["DTE"] = dte
    chain["Underlying"] = stockprice

    
    return chain

def get_bvd(symbol):
    """Assembles band, volataility and dividend dictionary"""
    
    b = get_bands(symbol)
    v = get_daily_volatility(symbol)
    d = get_dividend(symbol)
    s = {'Symbol': symbol}
    
    b.update(v)
    b.update(d)
    b.update(s)
    
    return pd.DataFrame([b])

### Make a dataframe of symbols and expiries

expiries = get_expiry_dates(expiry_url)
expiries['Key'] = 1

symbols = pd.DataFrame(get_symbols(symbols_url))
symbols['Key'] = 1

sym_expiry = pd.merge(expiries, symbols, on=['Key']).drop('Key', axis=1)

### Build the option chain table
sym_expiry = sym_expiry.loc[0:2]    # Limiting data
o = np.vectorize(get_option_chain)

df1 = pd.concat(o(sym_expiry.Symbol, sym_expiry['Expiry Date'], sym_expiry.DTE))
df1

### Make a dataframe of symbols with dividend, interest, bands and volatility
symbolz = symbols.loc[:, 'Symbol'][0:2]    #  Taking out the key
bvd = np.vectorize(get_bvd)

df2 = pd.concat(bvd(symbolz))

# Merge bvd with sym_expiry
df=df1.merge(df2, how='left', on='Symbol')

df.columns = ['cOI',  'cOIChng',  'cVolume',  'cIV',  'cLTP',  'cNetChng',  
              'cBidQty',  'cBidPrice',  'cAskPrice',  'cAskQty',  'Strike',  
              'pBidQty',  'pBidPrice',  'pAskPrice',  'pAskQty',  'pNetChng',  
              'pLTP',  'pIV',  'polume',  'pOIChng',  'pOI',  
              'Symbol',  'Expiry',  'DTE',  'Underlying', 'Margin',  
              'PriceHi_dt',  'PriceLo_dt',  'dailyVolatility',  'dividend',  
              'high52',  'low52',  'marketLot',  'PriceLo',  'PriceHi']

numcolumns = ['cOI',  'cOIChng',  'cVolume',  'cIV',  'cLTP',  'cNetChng',  
              'cBidQty',  'cBidPrice',  'cAskPrice',  'cAskQty',  'Strike',  
              'pBidQty',  'pBidPrice',  'pAskPrice',  'pAskQty',  'pNetChng',  
              'pLTP',  'pIV',  'polume',  'pOIChng',  'pOI',  
              'dailyVolatility',  'dividend', 'high52',  'low52',  
              'marketLot',  'PriceLo',  'PriceHi', 'Margin']


# convert numbers to numeric data
df[numcolumns] = df[numcolumns].apply(pd.to_numeric, errors = 'coerce')

# convert volatilities to percentages
df.loc[:, ['cIV', 'pIV', 'dailyVolatility', 'Margin']] = df.loc[:,['cIV', 'pIV', 'dailyVolatility', 'Margin']].apply(lambda x: x/100)

# convert dailyvolatility into annual volatility (for Black Scholes)
df.loc[:, 'dailyVolatility'] = df.loc[:,'dailyVolatility'].apply(lambda x: x*sqrt(num_trading_days_in_year))
# rename to volatility
df.rename(columns={'dailyVolatility' : 'Volatl'}, inplace=True)


Wall time: 4.85 s


In [119]:
list(df)

['cOI',
 'cOIChng',
 'cVolume',
 'cIV',
 'cLTP',
 'cNetChng',
 'cBidQty',
 'cBidPrice',
 'cAskPrice',
 'cAskQty',
 'Strike',
 'pBidQty',
 'pBidPrice',
 'pAskPrice',
 'pAskQty',
 'pNetChng',
 'pLTP',
 'pIV',
 'polume',
 'pOIChng',
 'pOI',
 'Symbol',
 'Expiry',
 'DTE',
 'Underlying',
 'Margin',
 'PriceHi_dt',
 'PriceLo_dt',
 'Volatl',
 'dividend',
 'high52',
 'low52',
 'marketLot',
 'PriceLo',
 'PriceHi']

In [42]:
# df.loc[:, ['cIV', 'pIV', 'dailyVolatility']].apply(lambda x: x/100)
df.loc[:, ['cIV', 'pIV', 'dailyVolatility']].dtypes

cIV                object
pIV                object
dailyVolatility    object
dtype: object

In [ ]:
df.loc[:, 'dailyVolatility'] = df['dailyVolatility'].astype(float)* sqrt(252)/100
df.loc[:, 'cIV'] = df.cIV.astype(float)/100

In [ ]:
def convert_to_percent(df):
    """Divides a column of strings into percent
    Args:
       df: The dataframe
       column_name: The column to convert
    
    Returns:
        A numeric series divided by 100.
    """
    
    pd.to_numeric(df, errors='coerce')/100


In [21]:
df[['cIV',
 'cLTP',
 'Strike',
 'pLTP',
 'pIV',
 'dailyVolatility',
 'Symbol',
 'Expiry',
 'DTE',
 'Margin',
 'PriceLo',
 'PriceHi']]

,cIV,cLTP,Strike,pLTP,pIV,dailyVolatility,Symbol,Expiry,DTE,Margin,PriceLo,PriceHi
0,-,-,1360.0,-,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
1,-,-,1380.0,-,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
2,-,-,1400.0,-,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
3,-,-,1420.0,-,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
4,-,-,1440.0,-,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
5,-,-,1460.0,1.00,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
6,-,-,1480.0,1.00,-,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
7,-,-,1500.0,4.90,26.19,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
8,-,85.00,1520.0,8.30,26.13,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"
9,36.59,64.90,1540.0,12.50,25.28,0.253992,ACC,28MAR2018,9,12.50,"1,442.85","1,763.45"


In [3]:
x = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_NSERaw.xlsx"
writer = pd.ExcelWriter(x)
df.to_excel(writer, 'options', index=False, freeze_panes=(1, 1))
writer.save()

In [5]:
get_bvd('INFY')

,Symbol,applicableMargin,cm_adj_high_dt,cm_adj_low_dt,dailyVolatility,dividend,high52,low52,marketLot,pricebandlower,pricebandupper
0,INFY,12.50,24-JAN-18,22-AUG-17,1.17,0.13,"1,221.05",860.00,600,"1,047.10","1,279.70"
